In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, shape, LineString
import json

%matplotlib inline

import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from methods import link_df_to_geojson
from methods import point_df_to_geojson
from methods import reproject

In [3]:
step6_output_folder = "../../data/interim/step6_gtfs/version_12/"
data_interim_dir = "../../data/interim/"

In [5]:
print('read link')
link_file = step6_output_folder + "link.feather"
link_df = pd.read_feather(link_file)

print('read node')
node_file = step6_output_folder + "node.geojson"
node_gdf = gpd.read_file(node_file)

print('read shape')
shape_file = step6_output_folder + "shape.geojson"
shape_gdf = gpd.read_file(shape_file)

read link
read node
read shape


In [7]:
link_gdf = pd.merge(link_df,
                    shape_gdf[["id", "geometry"]],
                   how = "left",
                   on = "id")

link_gdf = gpd.GeoDataFrame(link_gdf, geometry = link_gdf["geometry"],
                            crs={"init" : "epsg:4326"})

C:\Users\wangs1\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
geom_length = link_gdf[['geometry']].copy()
geom_length = geom_length.to_crs(epsg = 26915)
geom_length["length"] = geom_length.length

link_gdf["length"] = geom_length["length"]

In [9]:
node_gdf['X'] = node_gdf['geometry'].apply(lambda p: p.x)
node_gdf['Y'] = node_gdf['geometry'].apply(lambda p: p.y)
node_gdf['point'] = [list(xy) for xy in zip(node_gdf.X, node_gdf.Y)]
node_dict = dict(zip(node_gdf.model_node_id, node_gdf.point))
    
link_gdf['A_point'] = link_gdf['A'].map(node_dict)
link_gdf['B_point'] = link_gdf['B'].map(node_dict)
link_gdf['locationReferences'] = link_gdf.apply(lambda x: [{'sequence':1, 
                                                             'point': x['A_point'],
                                                             'distanceToNextRef':x['length'],
                                                            'bearing' : 0,
                                                             'intersectionId':x['fromIntersectionId']},
                                                                         {'sequence':2, 
                                                             'point': x['B_point'],
                                                             'intersectionId':x['toIntersectionId']}],
                                                   axis = 1)

In [11]:
shape_gdf["shape_id"] = range(1, 1+ len(shape_gdf))

shape_id_dict = dict(zip(shape_gdf.id, shape_gdf.shape_id))

link_gdf["shape_id"] = link_gdf["id"].map(shape_id_dict)

In [12]:
%%time

print("-------write out link shape geojson---------")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(shape_gdf, shape_prop)

with open(data_interim_dir + "step8_standard_format/shape.geojson", "w") as f:
    json.dump(shape_geojson, f)

-------write out link shape geojson---------
Wall time: 2min 50s


In [ ]:
shape_gdf.info()

In [14]:
%%time

# write out link variable json
# link unique handle "shstReferenceId" + "shstGeometryId"

print("-------write out link json---------")

link_prop = link_gdf.drop(['geometry', 'A_point', 'B_point', "service", "roundabout", 'est_width',
                          'highway', 'junction', 'key', 'landuse', 'link', 'area', 'id', 'length',
                          'width', 'bridge', 'tunnel',
                          'shape_id'], 
                          axis = 1).columns.tolist()

out = link_gdf[link_prop].to_json(orient = "records")

with open(data_interim_dir + "step8_standard_format/link.json", 'w') as f:
    f.write(out)

-------write out link json---------
Wall time: 33.5 s


In [15]:
print("-------write out link feather---------")

link_feather = link_gdf.drop(['geometry', 'A_point', 'B_point', "service", "roundabout", 'est_width',
                          'highway', 'junction', 'key', 'landuse', 'link', 'area', 'id', 'length',
                          'width', 'bridge', 'tunnel',
                             'shape_id'], 
                          axis = 1).copy()

link_feather.to_feather(data_interim_dir + 'step8_standard_format/link.feather')

-------write out link feather---------


C:\Users\wangs1\.conda\envs\MetC_new\lib\site-packages\ipykernel_launcher.py:9: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  if __name__ == '__main__':


In [16]:
link_feather.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1634790 entries, 0 to 1634789
Data columns (total 24 columns):
access                1632702 non-null object
bike_access           1634790 non-null int64
drive_access          1634790 non-null int64
fromIntersectionId    1632702 non-null object
lanes                 1632702 non-null object
maxspeed              1632702 non-null object
name                  1632702 non-null object
oneWay                1632702 non-null object
ref                   1632702 non-null object
roadway               1632702 non-null object
shstGeometryId        1634790 non-null object
shstReferenceId       1632702 non-null object
toIntersectionId      1632702 non-null object
u                     1632702 non-null float64
v                     1632702 non-null float64
walk_access           1634790 non-null int64
wayId                 1632702 non-null object
county                1634790 non-null object
model_link_id         1634790 non-null int64
A      

In [18]:
%%time

print("-------write out node geojson---------")

node_prop = node_gdf.drop(["geometry", "point"], axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(node_gdf, node_prop)

with open(data_interim_dir + "step8_standard_format/node.geojson", "w") as f:
    json.dump(node_geojson, f)

-------write out node geojson---------
Wall time: 2min 23s


In [17]:
node_gdf.info()

Index(['osm_node_id', 'shst_node_id', 'county', 'drive_access', 'walk_access',
       'bike_access', 'model_node_id', 'rail_only', 'geometry', 'X', 'Y',
       'point'],
      dtype='object')